# Carga del *Optical Recognition of Handwritten Digits Data Set*

Vamos a comenzar cargando nuestro Data Set que viene separado en dos archivos, ``optdigits.tra`` y ``opdigits.tes``. Estos archivos contienen los datos de entrenamiento y los datos de validación respectivamente. Utilizaremos ``pandas`` para cargar el Data Set y realizar las manipulaciones que resulten oportunas antes de proceder con el entrenamiento de las redes neuronales.

https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

In [1]:
import pandas as pd

# Utilizamos header = None para indicar a pandas que los ficheros no tienen
# ninguna cabezera
df_train = pd.read_csv('./optdigits.tra', header = None)
df_test = pd.read_csv('./optdigits.tes', header = None)

df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0,1,6,15,12,1,0,0,0,7,...,0,0,0,6,14,7,1,0,0,0
1,0,0,10,16,6,0,0,0,0,7,...,0,0,0,10,16,15,3,0,0,0
2,0,0,8,15,16,13,0,0,0,1,...,0,0,0,9,14,0,0,0,0,7
3,0,0,0,3,11,16,0,0,0,0,...,0,0,0,0,1,15,2,0,0,4
4,0,0,5,14,4,0,0,0,0,0,...,0,0,0,4,12,14,7,0,0,6


In [2]:
df_train.dtypes

0     int64
1     int64
2     int64
3     int64
4     int64
      ...  
60    int64
61    int64
62    int64
63    int64
64    int64
Length: 65, dtype: object

In [3]:
df_train[64].value_counts()

1    389
3    389
7    387
4    387
9    382
2    380
8    380
6    377
0    376
5    376
Name: 64, dtype: int64

En este caso, vemos como todas las variables de entrada son de tipo numérico por lo que no debemos reemplazar ninguna variable categórica. Puede resultar intersante sin embargo, realizar un escalado de los datos para que todos ellos queden en el rango (0, 1). Utilizaremos ``MinMaxScaler()``. En este caso, no es necesario el uso de un escalado robusto a *outliers* pues todos los valores de entrada se encuentran en el rango [0, 16].

Otro aspecto a comentar es que, si bien no todas las clases tienen el mismo número de elementos, las diferencias son poco significativas, por lo que estamos trabajando con un dataset equilibrado.

In [4]:
from sklearn.preprocessing import MinMaxScaler

# Separamos en valores de entrada y clases para training y test
X_tr = df_train[df_train.columns[:-1]]
y_tr = df_train[64]
X_te = df_test[df_test.columns[:-1]]
y_te = df_test[64]

# Realizamos el escalado de los datos de entrada
min_max = MinMaxScaler()
X_tr = min_max.fit_transform(X_tr)
X_te = min_max.transform(X_te)
X_tr

array([[0.    , 0.125 , 0.375 , ..., 0.0625, 0.    , 0.    ],
       [0.    , 0.    , 0.625 , ..., 0.1875, 0.    , 0.    ],
       [0.    , 0.    , 0.5   , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.    , 0.1875, ..., 0.5625, 0.    , 0.    ],
       [0.    , 0.    , 0.375 , ..., 1.    , 0.3125, 0.    ],
       [0.    , 0.    , 0.125 , ..., 0.    , 0.    , 0.    ]])

# Entrenamiento de modelos de redes neuronales con diferentes valores de sus hiperparámetros

A continuación vamos a entrenar y obtener las métricas de rendimiento de redes neuronales configuradas con diferentes valores de sus hiperparámetros. Los hiperparámetros que vamos a ajustar serán los siguientes:

- Función de activación
- Épocas de entrenamiento
- Tasa de entrenamiento
- Topología de la red

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

# Definimos las diferentes funciones de activación
f_acts = ['identity', 'logistic', 'relu', 'tanh']
# Definmos las diferentes topologías de la red (capas ocultas y neuronas por capa)
hidden_layers = [(x, ) for x in range(100, 300, 100)] + [(x, y) for x in range(100, 300, 100) for y in range(100, 300, 100)]
# Definimos las diferentes tasas de entrenamiento
learning_rates = np.array([0.0001, 0.001, 0.01, 0.1])
# Definimos las épocas de entrenamiento
epochs = np.array([250, 500, 750, 1000])

print(learning_rates)
print(epochs)

# Para cada experimento que realicemos iremos almacenando la accuracy del clasificador entrenado
accs = []
i, j, k = 0, 0, 0
# Recorremos las funciones de activación
for f in f_acts:
    accs.append([])
    # Recorremos las diferentes topologías
    for hl in hidden_layers:
        accs[i].append([])
        # Recorremos las iteraciones máximas permitidas
        for epoch in epochs:
            accs[i][j].append([])
            # Recorremos los diferentes learning rates
            for lr in learning_rates:
                # Entrenamos una red con la configuración actual
                model_temp = MLPClassifier(activation = f, max_iter = epoch, learning_rate_init = lr, hidden_layer_sizes = hl)
                model_temp.fit(X_tr, y_tr)
                # Generamos la predicción, medimos la accuracy y almacenamos el resultado
                y_temp = model_temp.predict(X_te)
                acc = accuracy_score(y_te, y_temp)
                print(f'Config actual --- f = {f} --- hidden_layers = {hl} --- epochs = {epoch} --- learning_rate = {lr} --- Accuracy = {acc}')
                accs[i][j][k].append(acc)
            k = k + 1
        j, k = j + 1, 0
    i, j = i + 1, 0
#print(accs)

In [6]:
accs_aux = np.array(accs)
i, j, k, m = np.unravel_index(accs_aux.argmax(), accs_aux.shape)
# Mostramos los parámetros óptimos para el experimento
print('--------------------------------------------------------')
print(f'CONFIGURACION OPTIMA\nAccuracy -> {accs_aux.max()}')
print(f'F. Activación -> {f_acts[i]}\nHidden Layers -> {hidden_layers[j]}\nIteraciones Máximas -> {epochs[k]}\nLearning Rate -> {learning_rates[m]}')
print('--------------------------------------------------------')

--------------------------------------------------------
CONFIGURACION OPTIMA
Accuracy -> 0.9716193656093489
F. Activación -> relu
Hidden Layers -> (200, 200)
Iteraciones Máximas -> 250
Learning Rate -> 0.01
--------------------------------------------------------


In [7]:
print(accs_aux)

[[[[0.94657763 0.95269894 0.95158598 0.89927657]
   [0.95158598 0.95102949 0.94824708 0.90428492]
   [0.9476906  0.95102949 0.94602115 0.90261547]
   [0.95158598 0.94824708 0.95158598 0.91819699]]

  [[0.94936004 0.94880356 0.94156928 0.89538119]
   [0.95102949 0.95214246 0.93934335 0.91819699]
   [0.94991653 0.95269894 0.93934335 0.90873678]
   [0.95047301 0.95158598 0.95102949 0.89705064]]

  [[0.94824708 0.95269894 0.93377852 0.92598776]
   [0.94936004 0.95436839 0.93155259 0.92765721]
   [0.95381191 0.95047301 0.93544797 0.92264886]
   [0.94991653 0.95158598 0.94156928 0.93656093]]

  [[0.95269894 0.94991653 0.934335   0.93489149]
   [0.95269894 0.95325543 0.87813022 0.91040623]
   [0.95214246 0.95158598 0.92932666 0.89705064]
   [0.94936004 0.94991653 0.9309961  0.89705064]]

  [[0.95269894 0.95381191 0.95158598 0.90651085]
   [0.95102949 0.95102949 0.92932666 0.94379521]
   [0.95214246 0.95102949 0.89371174 0.92988314]
   [0.95325543 0.95325543 0.91930996 0.92598776]]

  [[0.9526

De la salida anterior podemos observar como la configuración óptima (exactitud ~97.16%) se obtiene con los siguientes parámetros:

- Función de Activación: ReLU
- Topología de la red: 2 capas ocultas con 200 neuronas por capa (200, 200)
- Iteraciones máximas: 250 iteraciones
- Learning rate: 0.01

Sin embargo, en general podemos ver valores de exactitud en torno al 90% para la gran mayoría de configuraciones de los parámetros, lo que nos puede dar a entender que este tipo de modelo no es demasiado sensible a varaciones en sus hiperparámetros. Podemos destacar sin embargo algunos valores significativamente bajos de la precisión (en torno al 10% y al 20%) que ocurren cuando el parámetro ``learning_rate`` vale 0.1. 

El *learning rate* hace referencia a cuánto se actualizan los pesos de la red neuronal en cada una de las iteraciones que se calculan y tiene un impacto extremadamente significativo en el algoritmo de descenso de gradiente estocástico (el algoritmo que optmiza los pesos). Valores demasiado pequeños de este parámetro pueden provocar que el algoritmo oscile alrededor de un mínimo local y no consiga converger al mínimo global. Valores demasiado altos pueden desencadenar en el aprendizaje un un conjunto subóptimo de pesos, o un proceso de entrenamiento poco estable.

Para el problema que estamos tratando, un *learning rate* de 0.1 parece desencadenar en un proceso de aprendizaje subóptimo y en consecuencia, una exactitud muy baja. A continuación vamos a entrenar una red neuronal con los parámetros óptimos y obtendremos la *matriz de confusión* para comentar algunos aspectos que consideramos relevantes.

In [8]:
f_opt, top, ep, lrate = 'relu', (100, 100), 750, 0.01
model_opt = MLPClassifier(activation = f_opt, max_iter = ep, learning_rate_init = lrate, hidden_layer_sizes = top)
model_opt.fit(X_tr, y_tr)
y_pred = model_opt.predict(X_te)
conf_mat = confusion_matrix(y_te, y_pred)
df_aux = pd.DataFrame(conf_mat, index = list(range(0, 10)), columns = list(range(0, 10)))
df_aux.index.name, df_aux.columns.name = 'Real', 'Predicho'
df_aux

Predicho,0,1,2,3,4,5,6,7,8,9
Real,,,,,,,,,,
0,176,0,0,0,0,2,0,0,0,0
1,0,180,0,0,0,0,1,0,1,0
2,0,1,174,0,0,0,2,0,0,0
3,0,0,3,172,0,1,0,0,5,2
4,0,1,0,0,179,0,0,1,0,0
5,0,0,1,1,0,178,0,0,0,2
6,1,0,0,0,2,0,177,0,1,0
7,0,0,0,0,0,7,0,165,1,6
8,0,6,0,0,0,2,0,0,164,2


Como puede observarse a partir de la matriz de confusión anterior, la mayoría de patrones se predicen correctamente pues los valores más altos se concentran en la diagonal. Destacan sin embargo dos cifras un tanto más altas fuera de la diagonal. Por una parte, tenemos **6 valores que realmente eran un 8 y se han predicho como un 1** y, por otra parte, tenemos **7 valores que realmente eran un 7 y se han predicho como un 5** así como **6 valores que realmente eran un 7 y se han predicho como un 9**. La explicación de este fenómeno es que en muchas ocasiones estos dígitos manuscritos pueden resultar muy similares y sus transcripciones a imágenes normalizadas concentrarían los píxeles más oscuros en regiones muy similares. De ahí que, por muy potente que sea nuestro modelo, este error se muestra de forma recurrente (incluso un ser humano no podría distinguir estos dígitos en algunas circunstancias).